# CO2 analysis

This notebook is being used to prepare data and figures for the upcoming APERC Annual CO2 Emissions Report, due for publication in June 2020. The data and results here are strictly preliminary at this point.

A final interactive notebook will be made available at publication.

To use this notebook:
1. import the libraries below
2. import the prepared data
3. Plot all economies together
4. Plot each economy separately. You can select either fuels or sectors.

In [ ]:
import numpy as np
import pandas as pd
import glob
import os

import param
import hvplot.pandas

from bokeh.palettes import brewer
from bokeh.palettes import Category20
from bokeh.palettes import viridis
from bokeh.palettes import inferno
import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.models import Legend
from bokeh.plotting import figure

from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot

import panel as pn
pn.extension()
import panel.widgets as pnw
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvas
output_notebook()

In [ ]:
sns.set(style="ticks")

### Read in prepared data:

In [ ]:
df = pd.read_csv('../data/CO2 emissions prepared.csv')

In [ ]:
df = df.sort_values(by='Economy')
economies = df['Economy'].unique().tolist()
df = df.reset_index(drop=True)

### Plot all economies together:

In [ ]:
to_plot = ['Total CO2 Emission']
_df = df.set_index('Item')
_df = _df.loc[to_plot]

In [ ]:
df_pivot = _df.pivot_table(index=['Year'],columns='Economy',values='Value')
df_pivot = df_pivot.reset_index(drop=False)

In [ ]:
v = figure(width=1000, height=600,title='APEC',x_range=(1990, 2017)) 
names = df_pivot.drop('Year',axis=1).columns.values.tolist()
numlines=len(names)

v.varea_stack(names,x='Year', 
              color=viridis(numlines),
              alpha=0.7,
              muted_color=viridis(numlines), 
              muted_alpha=0.2,
              legend_label=names, 
              source=df_pivot)
v.legend.location = "top_left"
v.legend.click_policy="mute"

In [ ]:
show(v)

### Stacked area

In [ ]:
def stacked_area(df,things_to_plot,y_range):

    df = df.loc[df['Item'].isin(things_to_plot)]
    df = df.reset_index(drop=True)
    df_unstacked = df.set_index(['Economy', 'Year','Item']).unstack('Item')
    
    figures = []
    
    for economy in economies:
        _df = df_unstacked.loc[(economy,),]
        numlines=len(_df.columns)
        _df = _df.reset_index()
        _df = _df.droplevel(0,axis=1)
        _df = _df.rename({'':'Year'},axis=1)
        names = _df.drop('Year',axis=1).columns.values.tolist()
        
        v = figure(width=500, 
                   height=300,
                   title=economy,
                   x_range=(1990, 2017),
                   y_range=y_range
                  ) 
        v.varea_stack(names, x='Year', 
                      #color=Category20[numlines],
                      color=brewer["Accent"][numlines],
                      alpha=0.7,
                      #muted_color=Category20[numlines],
                      muted_color=brewer["Accent"][numlines],
                      muted_alpha=0.2,
                      legend_label=names, 
                      source=_df)
        v.legend.location = "top_left"
        v.legend.click_policy="mute"
        figures.append(v)
    
    show(gridplot(figures, ncols=3, plot_width=400, plot_height=300))

Two sets of plots will be created, one by fuels and one by sectors:

In [ ]:
fuels= ['1.1 Coal & Coal Products',
                 '1.2 Crude Oil & Petroleum Products',
                 '1.3 Gas']

In [ ]:
sectors= [#'2.1 Transformation Sector',
          '2.1.1 Main Activity Producer',
          '2.1.2 Autoproducers',
          '2.1.3 Gas Processing',
          '2.1.4 Loss & Own Use',
          #'2.2 Final Energy Consumption Sector'
          '2.2.1 Industry Sector',
          '2.2.2 Transport Sector',
          '2.2.3 Residential & Commercial',
          '2.2.4 Other'
          ]

#### Plot by fuels:

In [ ]:
stacked_area(df,fuels,(0,10000))

#### Plot by sector:

In [ ]:
stacked_area(df,sectors,(0,10000))

In [ ]:
things_to_plot = sectors
df = df.loc[df['Item'].isin(things_to_plot)]

In [ ]:
df.head(2)

In [ ]:
p = figure(title="Economy emissions by sector")

In [ ]:
color=brewer["Accent"][len(things_to_plot)]
d = dict(zip(things_to_plot,color))
df['Color'] = df['Item'].map(d)

In [ ]:
p.scatter(x='Year', y='Value', fill_color='Color', line_color='#000000', source=df, size=12)

In [ ]:
show(p)

In [ ]:
df1 = df.loc[(df['Item'].isin(things_to_plot)) & (df['Year']==2017)]

In [ ]:
plt.figure(figsize=(18,12))
sns.swarmplot(x="Item", y="Value", hue="Economy", data=df1)

In [ ]:
sns.relplot(x="Value", y="Item", hue="Economy", size="Value",
            sizes=(40, 400), alpha=.7, palette="muted",
            height=6,aspect=2, data=df1)

In [ ]:
plt.figure(figsize=(16,8))
sns.stripplot(x="Value", y="Item", hue="Economy",
              size = 10,
              data=df1, 
              dodge=True, 
              alpha=1, 
              zorder=1,
              jitter=1)

## Panel

In [ ]:
import panel as pn
pn.extension()
import panel.widgets as pnw
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvas

In [ ]:
df.head()

In [ ]:
df[df['Item']=='Total CO2 Emission']

In [ ]:
fuels= ['1.1 Coal & Coal Products',
        '1.2 Crude Oil & Petroleum Products',
        '1.3 Gas']

economies = df['Economy'].unique().tolist()

fuel_selector = pn.widgets.Select(name='Fuel', options=fuels)
show_legend = pn.widgets.Checkbox(name='Show legend')

int_range_slider = pn.widgets.IntRangeSlider(
    name='Year Slider',
    start=1990, end=2017, value=(1990, 2017), step=1)

economy_selector = pn.widgets.Select(name='Economy',options=economies)

title = '### Stock Price Dashboard'
subtitle = 'This dashboard allows you to select a company and date range to see stock prices.'

@pn.depends(fuel_selector.param.value,int_range_slider.param.value,show_legend.param.value)
def get_data(fuel_selector,int_range_slider,show_legend):
    start_date = int_range_slider[0]
    end_date = int_range_slider[1]
    mask = (df['Item'] == fuel_selector) & (df['Year'] > start_date) & (df['Year'] <= end_date)
    _df = df.loc[mask]
    
    df_pivot = _df.pivot_table(index=['Year'],columns='Economy',values='Value')
    df_pivot = df_pivot.reset_index(drop=False)
    
    v = figure(width=600, height=400,title='APEC') 
    names = df_pivot.drop('Year',axis=1).columns.values.tolist()
    numlines=len(names)
    
    v.varea_stack(names,x='Year', 
                  color=viridis(numlines),
                  alpha=0.7,
                  muted_color=viridis(numlines), 
                  muted_alpha=0.2,
                  legend_label=names,
                  #x_range=(start_date,end_date),
                  source=df_pivot)
    v.legend.location = "top_left"
    v.legend.click_policy="mute"
    
    return v

dashboard = pn.Row(pn.Column(fuel_selector,int_range_slider,show_legend,get_data))
dashboard

#### As a class (object)?

In [ ]:
fuels= ['1.1 Coal & Coal Products',
        '1.2 Crude Oil & Petroleum Products',
        '1.3 Gas']

sectors= ['2.1 Transformation Sector',
          '2.1.1 Main Activity Producer',
          '2.1.2 Autoproducers',
          '2.1.3 Gas Processing',
          '2.1.4 Loss & Own Use',
          '2.2 Final Energy Consumption Sector',
          '2.2.1 Industry Sector',
          '2.2.2 Transport Sector',
          '2.2.3 Residential & Commercial',
          '2.2.4 Other'
          ]

In [ ]:
class carbon(param.Parameterized):
    fuel_selector = param.ObjectSelector(default="1.1 Coal & Coal Products", objects=fuels)
    year_range = param.Range(default=(1990, 2017), bounds=(1990, 2017))
    show_legend = param.Boolean(True, doc="Show the legend")
    
    title = 'CO2 emissions'
    xlabel = 'Years'
    ylabel = 'Million tons'
    
    def get_data(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        mask = (df['Item'] == self.fuel_selector) & (df['Year'] > start_date) & (df['Year'] <= end_date)
        _df = df.loc[mask]
        
        df_pivot = _df.pivot_table(index=['Year'],columns='Economy',values='Value')
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot
       
    def hv_plot_bar(self):
        data  = self.get_data()
        title = 'Bar chart'
        names = data.drop('Year',axis=1).columns.values.tolist()
        
        plot  = data.hvplot.bar(x ='Year', y=names, title=title,width=400, height=400, legend=False, alpha=0.4, stacked=True, rot=90)
        #plot  = plot.options(legend_position='top')
        return plot
    
    def hv_plot_area(self):
        data  = self.get_data()
        title = 'Area chart'
        names = data.drop('Year',axis=1).columns.values.tolist()
        
        plot  = data.hvplot.area(x ='Year', y=names, title=title,width=500, height=400, legend=self.show_legend,alpha=0.4)
        #plot  = plot.options(legend_position='bottom_right')
        return plot
    
c = carbon(name='')

In [ ]:
pn.Column(pn.Row(c.param,c.hv_plot_bar, c.hv_plot_area))